In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
# Set image dimensions and dataset directories
img_height, img_width = 224, 224
train_dir = '/kaggle/input/hackathon-online-thai-silk-pattern-recognition/Train/Train'  # Update this with the correct path to your 'Train' folder

In [ ]:
# Preprocess images using ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Use the CutMix data generator
cutmix_train_generator = cutmix_data_generator(train_generator, alpha=1.0)

In [ ]:
train_generator.class_indices

In [ ]:
# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Set base_model layers as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Build a new model on top of the base_model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(cutmix_train_generator, validation_data=val_generator, epochs=10)

# Save the model
model.save('thai_silk_pattern_recognition_transfer_learning.h5')

In [ ]:
submission = pd.read_csv('/kaggle/input/hackathon-online-thai-silk-pattern-recognition/test.csv')
images_dir = '/kaggle/input/hackathon-online-thai-silk-pattern-recognition/Test/Test'

In [ ]:
submission.head()

In [ ]:
# Define the class mapping
class_mapping = {0: 1, 1: 2, 2: 3, 3: 4}

In [ ]:
# Loop through each row in the submission file
for index, row in submission.iterrows():
    img_id = row['id']
    
    # Load the image and preprocess it
    img_path = os.path.join(images_dir, f'{img_id}.jpeg')
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    # Make a prediction using the model
    preds = model.predict(x)
    
    # Get the label with the highest probability
    predicted_label = np.argmax(preds)
    
    # Map the integer label to the corresponding style name
    predicted_style = class_mapping[predicted_label]
    
    # Update the submission file with the predicted style name
    submission.loc[index, 'label'] = predicted_style

In [ ]:
submission.head(5)

In [ ]:
# Save the updated submission file
submission.to_csv('submission_VGG16.csv', index=False)